In [1]:
!git clone https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes.git

In [2]:
!ls ./histopath_failure_modes

In [3]:
!pip install deconvolution

In [4]:
# import staintools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm
import tifffile
import sys
import os
import numpy as np
from scipy import interpolate
from scipy.ndimage import morphology
from scipy.stats import multivariate_normal
from PIL import Image, ImageDraw
from cv2 import GaussianBlur, blur, getPerspectiveTransform, warpPerspective

from hashlib import blake2s

# from deconvolution import Deconvolution
# import deconvolution.pixeloperations as po


sys.path.append("/kaggle/working/histopath_failure_modes")
from image_manipulation import img_manip

In [5]:
!mkdir -p ./hubmap-aug-bubble/bubble_original_data

In [6]:
from joblib import Parallel, delayed
import joblib as joblib
import time

In [7]:
dest_dir1 = "./hubmap-aug-bubble/bubble_original_data"
DIR1 = "../input/hubmap-organ-segmentation/train_images"

def bubble_aug(dir_path, dest_dir1, idx):
    
    fname = os.listdir(dir_path)[idx]
    path = os.path.join(dir_path, fname)

    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)

    img_with_bubbles = img_manip.add_bubbles(im_pil, alpha=0.55, maxWidth = 120, nBubbles = 150)

    fpath = os.path.join(dest_dir1, fname)

    img_with_bubbles.save(fpath)

    return

start_time = time.time()
number_of_cpu = joblib.cpu_count()

delayed_funcs = [delayed(bubble_aug)(DIR1,dest_dir1, i) for i in range(len(os.listdir(DIR1)))]
parallel_pool = Parallel(n_jobs=number_of_cpu)

parallel_pool(delayed_funcs)
print("program took time: ", time.time() - start_time)

In [10]:
!mkdir -p ./hubmap-aug-bubble/bubble_pixel_size_fixed_data

In [11]:
dest_dir2 = "./hubmap-aug-bubble/bubble_pixel_size_fixed_data"
DIR2 = "../input/hubmap-22-aug-pixel-size"

start_time = time.time()
number_of_cpu = joblib.cpu_count()

delayed_funcs = [delayed(bubble_aug)(DIR2, dest_dir2, i) for i in range(len(os.listdir(DIR2)))]
parallel_pool = Parallel(n_jobs=number_of_cpu)

parallel_pool(delayed_funcs)
print("program took time: ", time.time() - start_time)

In [12]:
!ls ./hubmap-aug-bubble/bubble_pixel_size_fixed_data -la